In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd
import time

# 25년1월20일 ~ 26년 1월 22일
def scrape_presidency_only(
    url="https://rollcall.com/factbase/trump/search/",
    max_scrolls=300,
    sleep_after_scroll=1.8,
    no_growth_limit=8,            # results 증가 없으면 종료(연속 n회)
    checkpoint_every=300,         # n건마다 임시 저장
    out_csv="trump_presidency_only.csv",
    checkpoint_csv="trump_presidency_only_partial.csv",
):
    # -----------------------------
    # [1] 대통령 임기 판별
    # -----------------------------
    term1_start = datetime(2017, 1, 20)
    term1_end   = datetime(2021, 1, 20)
    term2_start = datetime(2025, 1, 20)

    def is_president_term(date_obj: datetime) -> bool:
        if term1_start <= date_obj <= term1_end:
            return True
        if date_obj >= term2_start:
            return True
        return False

    # -----------------------------
    # [2] 브라우저 설정
    # -----------------------------
    opt = webdriver.ChromeOptions()
    opt.add_argument("--headless=new")
    opt.add_argument("--no-sandbox")
    opt.add_argument("--disable-dev-shm-usage")
    opt.add_argument("--window-size=1920,1080")
    opt.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opt)

    results = []
    processed_urls = set()

    # 종료 제어용
    no_growth = 0
    prev_results_len = 0

    try:
        print(f"🌐 접속 중: {url}")
        driver.get(url)

        # 페이지가 로드될 때까지 대기(너무 느리면 15초까지)
        WebDriverWait(driver, 15).until(lambda d: d.execute_script("return document.readyState") == "complete")
        time.sleep(2)

        print("⬇️ 스크롤을 내리며 '임기 기간' 데이터만 수집합니다...")

        scroll_count = 0
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 임기 이전(2017-01-20 이전) 날짜를 발견했는지 기록
        seen_pre_2017 = False

        while scroll_count < max_scrolls:
            soup = BeautifulSoup(driver.page_source, "html.parser")

            title_divs = soup.select('div[x-text="item.record_title"]')
            link_btns  = soup.select('a[title="View Transcript"]')

            min_len = min(len(title_divs), len(link_btns))

            # 이번 페이지에서 파싱된 날짜 중 "가장 오래된 날짜" 추적(종료 판단용)
            oldest_dt_in_view = None

            for i in range(min_len):
                link = link_btns[i].get("href")
                if not link:
                    continue
                if "rollcall.com" not in link:
                    link = "https://rollcall.com" + link

                if link in processed_urls:
                    continue
                processed_urls.add(link)

                full_text = title_divs[i].get_text(strip=True)

                # 보통 "TITLE - January 21, 2026" 형태
                if " - " not in full_text:
                    continue

                date_str = full_text.split(" - ")[-1].strip()
                try:
                    dt_obj = datetime.strptime(date_str, "%B %d, %Y")
                except ValueError:
                    continue

                # oldest 추적
                if oldest_dt_in_view is None or dt_obj < oldest_dt_in_view:
                    oldest_dt_in_view = dt_obj

                # ✅ 임기 기간이면 저장
                if is_president_term(dt_obj):
                    results.append({
                        "date": date_str,
                        "url": link,
                        "title": full_text.split(" - ")[0].strip()
                    })

                    # ✅ 체크포인트 저장(중간 저장)
                    if checkpoint_every and len(results) % checkpoint_every == 0:
                        pd.DataFrame(results).drop_duplicates(subset=["url"]).to_csv(
                            checkpoint_csv, index=False, encoding="utf-8-sig"
                        )
                        print(f"💾 [중간저장] {checkpoint_csv} | 현재 {len(results)}건")

            # -----------------------------
            # [A] 종료조건 1: 임기 이전으로 내려왔으면 종료
            # -----------------------------
            # 화면에 보이는 것 중 가장 오래된 날짜가 2017-01-20 이전이면
            # 더 내려가도 임기 데이터는 절대 안 나오므로 종료
            if oldest_dt_in_view is not None and oldest_dt_in_view < term1_start:
                print(f"🛑 임기 시작({term1_start.date()}) 이전 날짜({oldest_dt_in_view.date()}) 도달 → 종료")
                seen_pre_2017 = True
                break

            # -----------------------------
            # [B] 종료조건 2: results 증가가 없으면 종료
            # -----------------------------
            if len(results) == prev_results_len:
                no_growth += 1
            else:
                no_growth = 0
                prev_results_len = len(results)

            if no_growth >= no_growth_limit:
                print(f"🛑 임기 데이터 증가 없음({no_growth_limit}회 연속) → 종료")
                break

            # -----------------------------
            # [C] 스크롤 다운 + 바닥 체크
            # -----------------------------
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(sleep_after_scroll)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("🎉 페이지 바닥 도달(더 이상 로드 없음) → 종료")
                break

            last_height = new_height
            scroll_count += 1

            if scroll_count % 10 == 0:
                print(f"   ... 스크롤 {scroll_count}회 | 수집된 데이터: {len(results)}건 | no_growth={no_growth}")

    except Exception as e:
        print(f"❌ 에러: {e}")

    finally:
        driver.quit()

    # -----------------------------
    # [4] 최종 저장
    # -----------------------------
    if not results:
        print("⚠️ 조건(임기 기간)에 맞는 데이터가 없습니다.")
        return None

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])

    # 날짜 정렬(최신순)
    df["_dt"] = pd.to_datetime(df["date"], format="%B %d, %Y", errors="coerce")
    df = df.sort_values("_dt", ascending=False).drop(columns=["_dt"])

    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ 최종 저장 완료: {out_csv} (총 {len(df)}건)")
    print(f"   (참고) processed_urls={len(processed_urls)} | seen_pre_2017={seen_pre_2017}")
    return df


# 실행
df_presidency = scrape_presidency_only()
if df_presidency is not None:
    print(df_presidency.head())


🌐 접속 중: https://rollcall.com/factbase/trump/search/
⬇️ 스크롤을 내리며 '임기 기간' 데이터만 수집합니다...
💾 [중간저장] trump_presidency_only_partial.csv | 현재 300건
   ... 스크롤 10회 | 수집된 데이터: 397건 | no_growth=0
💾 [중간저장] trump_presidency_only_partial.csv | 현재 600건
   ... 스크롤 20회 | 수집된 데이터: 792건 | no_growth=0
💾 [중간저장] trump_presidency_only_partial.csv | 현재 900건
   ... 스크롤 30회 | 수집된 데이터: 976건 | no_growth=5
🛑 임기 데이터 증가 없음(8회 연속) → 종료

✅ 최종 저장 완료: trump_presidency_only.csv (총 976건)
   (참고) processed_urls=1320 | seen_pre_2017=False
                date                                                url  \
12  January 17, 2027  https://rollcall.com/factbase/trump/transcript...   
0   January 21, 2026  https://rollcall.com/factbase/trump/transcript...   
5   January 21, 2026  https://rollcall.com/factbase/trump/transcript...   
6   January 21, 2026  https://rollcall.com/factbase/trump/transcript...   
1   January 21, 2026  https://rollcall.com/factbase/trump/transcript...   

                                            

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime
import pandas as pd
import time

def scrape_term1_only(
    url="https://rollcall.com/factbase/trump/search/",
    max_scrolls=600,
    sleep_after_scroll=1.8,
    no_growth_limit=10,
    checkpoint_every=300,
    out_csv="trump_term1_only.csv",
    checkpoint_csv="trump_term1_only_partial.csv",
):
    term1_start = datetime(2017, 1, 20)
    term1_end   = datetime(2021, 1, 20)

    def in_term1(dt_obj: datetime) -> bool:
        return term1_start <= dt_obj <= term1_end

    opt = webdriver.ChromeOptions()
    opt.add_argument("--headless=new")
    opt.add_argument("--no-sandbox")
    opt.add_argument("--disable-dev-shm-usage")
    opt.add_argument("--window-size=1920,1080")
    opt.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opt)

    results = []
    processed_urls = set()

    no_growth = 0
    prev_processed_len = 0

    try:
        driver.get(url)
        WebDriverWait(driver, 15).until(lambda d: d.execute_script("return document.readyState") == "complete")
        time.sleep(2)

        scroll_count = 0
        last_height = driver.execute_script("return document.body.scrollHeight")

        while scroll_count < max_scrolls:
            soup = BeautifulSoup(driver.page_source, "html.parser")
            title_divs = soup.select('div[x-text="item.record_title"]')
            link_btns  = soup.select('a[title="View Transcript"]')
            min_len = min(len(title_divs), len(link_btns))

            oldest_dt_in_view = None

            for i in range(min_len):
                link = link_btns[i].get("href")
                if not link:
                    continue
                if "rollcall.com" not in link:
                    link = "https://rollcall.com" + link

                if link in processed_urls:
                    continue
                processed_urls.add(link)

                full_text = title_divs[i].get_text(strip=True)
                if " - " not in full_text:
                    continue

                date_str = full_text.split(" - ")[-1].strip()
                try:
                    dt_obj = datetime.strptime(date_str, "%B %d, %Y")
                except ValueError:
                    continue

                if oldest_dt_in_view is None or dt_obj < oldest_dt_in_view:
                    oldest_dt_in_view = dt_obj

                # ✅ 1기만 저장
                if in_term1(dt_obj):
                    results.append({
                        "date": date_str,
                        "url": link,
                        "title": full_text.split(" - ")[0].strip()
                    })

                    if checkpoint_every and len(results) % checkpoint_every == 0:
                        pd.DataFrame(results).drop_duplicates(subset=["url"]).to_csv(
                            checkpoint_csv, index=False, encoding="utf-8-sig"
                        )
                        print(f"💾 [중간저장] {checkpoint_csv} | 현재 {len(results)}건")

            # ✅ 종료조건 A: 2017-01-20 이전으로 내려가면 1기 끝난 것
            if oldest_dt_in_view is not None and oldest_dt_in_view < term1_start:
                print(f"🛑 1기 시작({term1_start.date()}) 이전({oldest_dt_in_view.date()}) 도달 → 종료")
                break

            # ✅ 종료조건 B: results가 아니라 processed_urls 증가로 판단 (중요!)
            if len(processed_urls) == prev_processed_len:
                no_growth += 1
            else:
                no_growth = 0
                prev_processed_len = len(processed_urls)

            if no_growth >= no_growth_limit:
                print(f"🛑 더 이상 새 항목 로드 없음({no_growth_limit}회) → 종료")
                break

            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(sleep_after_scroll)

            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                print("🎉 페이지 바닥 도달 → 종료")
                break

            last_height = new_height
            scroll_count += 1

            if scroll_count % 10 == 0:
                print(f"... 스크롤 {scroll_count}회 | term1 수집: {len(results)} | processed={len(processed_urls)}")

    finally:
        driver.quit()

    if not results:
        print("⚠️ 1기 기간 데이터가 없습니다.")
        return None

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    df["_dt"] = pd.to_datetime(df["date"], format="%B %d, %Y", errors="coerce")
    df = df.sort_values("_dt", ascending=False).drop(columns=["_dt"])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print(f"✅ 저장 완료: {out_csv} (총 {len(df)}건)")
    return df

df_term1 = scrape_term1_only()
print(df_term1.head())



... 스크롤 10회 | term1 수집: 0 | processed=400
... 스크롤 20회 | term1 수집: 0 | processed=800
... 스크롤 30회 | term1 수집: 0 | processed=1200
... 스크롤 40회 | term1 수집: 0 | processed=1600
... 스크롤 50회 | term1 수집: 199 | processed=2000
💾 [중간저장] trump_term1_only_partial.csv | 현재 300건
... 스크롤 60회 | term1 수집: 599 | processed=2400
💾 [중간저장] trump_term1_only_partial.csv | 현재 600건
💾 [중간저장] trump_term1_only_partial.csv | 현재 900건
... 스크롤 70회 | term1 수집: 998 | processed=2800
💾 [중간저장] trump_term1_only_partial.csv | 현재 1200건
... 스크롤 80회 | term1 수집: 1398 | processed=3200
💾 [중간저장] trump_term1_only_partial.csv | 현재 1500건
... 스크롤 90회 | term1 수집: 1797 | processed=3600
💾 [중간저장] trump_term1_only_partial.csv | 현재 1800건
💾 [중간저장] trump_term1_only_partial.csv | 현재 2100건
... 스크롤 100회 | term1 수집: 2197 | processed=4000
💾 [중간저장] trump_term1_only_partial.csv | 현재 2400건
🛑 1기 시작(2017-01-20) 이전(2016-12-07) 도달 → 종료
✅ 저장 완료: trump_term1_only.csv (총 2507건)
               date                                                url  \
0  January

In [5]:
df_term1.to_csv('/home/hyuksu/teamproject2/data/트럼프1')

In [11]:
import pandas as pd
a1=pd.read_csv('/home/hyuksu/teamproject2/data/트럼프1')
a2=pd.read_csv('/home/hyuksu/teamproject2/data/트럼프2')


In [37]:
a1.columns,a2.columns

(Index(['date', 'url', 'title'], dtype='str'),
 Index(['date', 'url', 'title'], dtype='str'))

In [21]:
a1.drop(columns='Unnamed: 0',inplace=True)
a2.drop(columns='Unnamed: 0',inplace=True)

In [36]:
len(a1),len(a2)

(2507, 976)

In [29]:
b=pd.concat([a2,a1],axis=0)


In [30]:
b.dropna(inplace=True)
len(b)

3483

In [32]:
b.drop(index=0,inplace=True)

In [34]:
b.to_csv('/home/hyuksu/teamproject2/data/total_url')

In [35]:
b['url'].head(10)

1     https://rollcall.com/factbase/trump/transcript...
2     https://rollcall.com/factbase/trump/transcript...
3     https://rollcall.com/factbase/trump/transcript...
4     https://rollcall.com/factbase/trump/transcript...
5     https://rollcall.com/factbase/trump/transcript...
6     https://rollcall.com/factbase/trump/transcript...
7     https://rollcall.com/factbase/trump/transcript...
8     https://rollcall.com/factbase/trump/transcript...
9     https://rollcall.com/factbase/trump/transcript...
10    https://rollcall.com/factbase/trump/transcript...
Name: url, dtype: str